In [21]:
import os
from glob import glob

import pandas as pd
import numpy as np
from collections import Counter

import pickle
import gc

from tqdm import tqdm
tqdm.pandas()

C:\Users\_v010ch_\Anaconda3\lib\site-packages\tqdm\std.py:697: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [22]:
import catboost as cb

In [64]:
DATA = './data'
DATA_OWN = './data_own'
CLICKSTREAM = 'alfabattle2_abattle_clickstream'
MODELS = './models'
UTILS = './utils'
SUBM = './submissions'

In [3]:
data = pd.read_csv(os.path.join(DATA, 'alfabattle2_prediction_session_timestamp.csv'), parse_dates=['timestamp'])
data.head()

,client_pin,timestamp
0,f0c674b2bb4dc64be607029271d706ec,2020-08-01 00:00:53
1,90725b54ce77576883813d87749df6bd,2020-08-01 00:02:57
2,eb0f82d74c7b7bd5eafbd5b5f8cb3e2a,2020-08-01 00:03:14
3,831bf4c0ecccc55e536b8cfb9153d672,2020-08-01 00:04:44
4,3f1a5a1c492ce877af833113e59c5797,2020-08-01 00:06:40


In [5]:
subm = pd.read_csv(os.path.join(DATA, 'alfabattle2_abattle_sample_prediction.csv'))
subm.head()

,client_pin,prediction
0,f0c674b2bb4dc64be607029271d706ec,credit_info
1,90725b54ce77576883813d87749df6bd,credit_info
2,eb0f82d74c7b7bd5eafbd5b5f8cb3e2a,own_transfer
3,831bf4c0ecccc55e536b8cfb9153d672,statement
4,3f1a5a1c492ce877af833113e59c5797,invest


In [15]:
def get_time_of_day(inp_hour):
    if (inp_hour >= 12) and (inp_hour < 18):
        return 'day'
    elif (inp_hour >= 6) and (inp_hour < 12):
        return 'morning'
    elif (inp_hour >= 18) and (inp_hour <= 23):
        return 'evening'
    else:
        return 'night'

In [16]:
%%time
data['dom']  = data.timestamp.apply(lambda x: x.day)
data['dow']  = data.timestamp.apply(lambda x: x.weekday())
data['hour'] = data.timestamp.apply(lambda x: x.hour)
data['tod']  = data.hour.apply(get_time_of_day)

Wall time: 842 ms


In [19]:
data.head()

,client_pin,timestamp,dom,dow,hour,tod
0,f0c674b2bb4dc64be607029271d706ec,2020-08-01 00:00:53,1,5,0,night
1,90725b54ce77576883813d87749df6bd,2020-08-01 00:02:57,1,5,0,night
2,eb0f82d74c7b7bd5eafbd5b5f8cb3e2a,2020-08-01 00:03:14,1,5,0,night
3,831bf4c0ecccc55e536b8cfb9153d672,2020-08-01 00:04:44,1,5,0,night
4,3f1a5a1c492ce877af833113e59c5797,2020-08-01 00:06:40,1,5,0,night


loading LabelBinarizer

In [93]:
lb_dom  = pickle.load(open((os.path.join(UTILS, 'lb_dom.pkl')),  'rb'))
lb_dow  = pickle.load(open((os.path.join(UTILS, 'lb_dow.pkl')),  'rb'))
lb_hour = pickle.load(open((os.path.join(UTILS, 'lb_hour.pkl')), 'rb'))
lb_tod  = pickle.load(open((os.path.join(UTILS, 'lb_tod.pkl')),  'rb'))

In [94]:
dom_features  = ['dom_'  + str(el) for el in lb_dom.classes_]
dow_features  = ['dow_'  + str(el) for el in lb_dow.classes_]
hour_features = ['hour_' + str(el) for el in lb_hour.classes_]
tod_features  = ['tod_'  + str(el) for el in lb_tod.classes_]

merge data with statistics

In [46]:
client_freq_targ = pd.read_csv(os.path.join(DATA_OWN, 'client_freq_targ.csv'))
data = data.merge(client_freq_targ, how= 'left', on='client_pin', validate='many_to_one')

In [47]:
client_freq_features = ['client_freq_main_screen', 'client_freq_statement',
       'client_freq_credit_info', 'client_freq_own_transfer',
       'client_freq_mobile_recharge', 'client_freq_phone_money_transfer',
       'client_freq_card2card_transfer', 'client_freq_chat',
       'client_freq_card_recharge', 'client_freq_invest']

In [25]:
glob_freq_dom  = pd.read_csv(os.path.join(DATA_OWN, 'glob_freq_dom.csv'))
glob_freq_dow  = pd.read_csv(os.path.join(DATA_OWN, 'glob_freq_dow.csv'))
glob_freq_hour = pd.read_csv(os.path.join(DATA_OWN, 'glob_freq_hour.csv'))
glob_freq_tod  = pd.read_csv(os.path.join(DATA_OWN, 'glob_freq_tod.csv'))

In [26]:
glob_freq_dom.columns  = ['dom_'  + el for el in glob_freq_dom.keys()]
glob_freq_dow.columns  = ['dow_'  + el for el in glob_freq_dow.keys()]
glob_freq_hour.columns = ['hour_' + el for el in glob_freq_hour.keys()]
glob_freq_tod.columns  = ['tod_'  + el for el in glob_freq_tod.keys()]

In [27]:
glob_freq_dom = glob_freq_dom.rename(  columns={'dom_dom': 'dom'})
glob_freq_dow = glob_freq_dow.rename(  columns={'dow_dow': 'dow'})
glob_freq_hour = glob_freq_hour.rename(columns={'hour_hour': 'hour'})
glob_freq_tod = glob_freq_tod.rename(  columns={'tod_tod': 'tod'})

#glob_freq_dom.keys(), glob_freq_dow.keys(), glob_freq_hour.keys(), glob_freq_tod.keys(), 

In [28]:
dom_freq_features  = [el for el in glob_freq_dom.keys()[1:]]
dow_freq_features  = [el for el in glob_freq_dow.keys()[1:]]
hour_freq_features = [el for el in glob_freq_hour.keys()[1:]]
tod_freq_features  = [el for el in glob_freq_tod.keys()[1:]]

In [29]:
data.shape

(79268, 6)

In [30]:
data = data.merge(glob_freq_dom, how= 'left', on='dom', validate='many_to_one')
data = data.merge(glob_freq_dow, how= 'left', on='dow', validate='many_to_one')
data = data.merge(glob_freq_hour, how= 'left', on='hour', validate='many_to_one')
data = data.merge(glob_freq_tod, how= 'left', on='tod', validate='many_to_one')
data.shape

(79268, 46)

## load models

In [55]:
clf_sgd = pickle.load(open(os.path.join(MODELS, 'clf_sgd.pkl'), 'rb'))
#clf_mlp = pickle.loads(os.paht.join(MODELS, 'clf_mlp.pkl'))
#clf_knn = pickle.loads(os.paht.join(MODELS, 'clf_knn.pkl'))
#clf_svc = pickle.loads(os.paht.join(MODELS, 'clf_svc.pkl'))
#clf_rf  = pickle.loads(os.paht.join(MODELS, 'clf_rf.pkl'))
#clf_ab  = pickle.loads(os.paht.join(MODELS, 'clf_ab.pkl'))
#clf_gaus= pickle.loads(os.paht.join(MODELS, 'clf_gauss.pkl'))

#clf_cb  = cb.load_model(os.paht.join(MODELS, 'clf_cb.cbm'), format='cbm')

## make predictions

In [56]:
using_features = \
                 client_freq_features + \
                 dom_freq_features + \
                 dow_freq_features + \
                 hour_freq_features + \
                 tod_freq_features 


                 # dates as a features \
                 #dom_features + \
                 #dow_features + \
                 #hour_features + \
                 #tod_features 
                 #
print(using_features)

['client_freq_main_screen', 'client_freq_statement', 'client_freq_credit_info', 'client_freq_own_transfer', 'client_freq_mobile_recharge', 'client_freq_phone_money_transfer', 'client_freq_card2card_transfer', 'client_freq_chat', 'client_freq_card_recharge', 'client_freq_invest', 'dom_glob_freq_main_screen', 'dom_glob_freq_statement', 'dom_glob_freq_credit_info', 'dom_glob_freq_own_transfer', 'dom_glob_freq_mobile_recharge', 'dom_glob_freq_phone_money_transfer', 'dom_glob_freq_card2card_transfer', 'dom_glob_freq_chat', 'dom_glob_freq_card_recharge', 'dom_glob_freq_invest', 'dow_glob_freq_main_screen', 'dow_glob_freq_statement', 'dow_glob_freq_credit_info', 'dow_glob_freq_own_transfer', 'dow_glob_freq_mobile_recharge', 'dow_glob_freq_phone_money_transfer', 'dow_glob_freq_card2card_transfer', 'dow_glob_freq_chat', 'dow_glob_freq_card_recharge', 'dow_glob_freq_invest', 'hour_glob_freq_main_screen', 'hour_glob_freq_statement', 'hour_glob_freq_credit_info', 'hour_glob_freq_own_transfer', 'ho

In [57]:
pred_sgd = clf_sgd.predict(data[using_features])

In [39]:
#pred_mlp = clf_mlp.predict(data[using_features])

In [40]:
#pred_knn = clf_knn.predict(data[using_features])

In [41]:
#pred_svc = clf_svc.predict(data[using_features])

In [42]:
#pred_rf = clf_rf.predict(data[using_features])

In [43]:
#pred_ab = clf_ab.predict(data[using_features])

In [44]:
#pred_gaus = clf_gau.predict(data[using_features])

In [45]:
#pred_cb = clf_cb.predict(data[using_features])

## make submit

In [99]:
subm = pd.read_csv(os.path.join(DATA, 'alfabattle2_abattle_sample_prediction.csv'))

In [105]:
subm.to_csv(os.path.join(SUBM, 'subm_client_4glob_sgd.csv'), index = False)